Import Libraries

In [3]:
import pandas as pd
import math


Import csv

In [4]:
df = pd.read_csv('modelinput.csv')

Helper Function to calculate distances

In [5]:
def calculate_distance(x1, y1, x2, y2):

    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

Create New Data Frame and fill in rows

In [8]:
# Initialize variables and DataFrames
final_df = pd.DataFrame()
new_row1 = {}
new_row2 = {}
new_row3 = {}
begun = False
play_start = False
count = 0

# Iterate through rows in df
for index, row in df.iterrows():
    play_id = row['play_id']
    if row['event_code'] == 1:
        play_start = True
    
    if play_start:
        count +=1
        if not begun:
            if row['event_code'] == 4:
                begun = True
                start_time = row['timestamp']
                starting_places = eval(row['player_positions'])
                
        elif row['event_code'] != 4000:
            ball_pos = eval(row['ball_pos'])

            if isinstance(ball_pos, tuple) and isinstance(starting_places, list) and len(starting_places)==3:
                ball_x, ball_y = ball_pos[0], ball_pos[1]
                # print(ball_pos)
                # print(starting_places)
                # print(play_id)
                # print(row['index'])
                distance_1 = calculate_distance(ball_x, ball_y, starting_places[0][7][0], starting_places[0][7][1])
                distance_2 = calculate_distance(ball_x, ball_y, starting_places[1][8][0], starting_places[1][8][1])
                distance_3 = calculate_distance(ball_x, ball_y, starting_places[2][9][0], starting_places[2][9][1])

                if row['event_code'] == 2:
                    catch_positions = eval(row['catch_position'])
                    new_row1 = {'distance': distance_1, 'time': row['timestamp'] - start_time, 'catch': catch_positions.get(7, 0)}
                    new_row2 = {'distance': distance_2, 'time': row['timestamp'] - start_time, 'catch': catch_positions.get(8, 0)}
                    new_row3 = {'distance': distance_3, 'time': row['timestamp'] - start_time, 'catch': catch_positions.get(9, 0)}
                else:
                    new_row1 = {'distance': distance_1, 'time': row['timestamp'] - start_time, 'catch': 0}
                    new_row2 = {'distance': distance_2, 'time': row['timestamp'] - start_time, 'catch': 0}
                    new_row3 = {'distance': distance_3, 'time': row['timestamp'] - start_time, 'catch': 0}

                # Convert dictionaries to DataFrames
                df_new_row_1 = pd.DataFrame(new_row1, index=[0])
                df_new_row_2 = pd.DataFrame(new_row2, index=[0])
                df_new_row_3 = pd.DataFrame(new_row3, index=[0])

                # Concatenate DataFrames to final_df
                final_df = pd.concat([final_df, df_new_row_1, df_new_row_2, df_new_row_3], ignore_index=True)

                # Reset new_row dictionaries
                new_row1 = {}
                new_row2 = {}
                new_row3 = {}

            begun = False
            play_start = False

print(count)

300846


Export as csv

In [7]:
print (final_df)

         distance  time  catch
0      133.321061  5500      0
1       23.774403  5500      1
2      163.004893  5500      0
3       59.623193  5150      1
4      162.019367  5150      0
...           ...   ...    ...
10525  188.289394  6300      0
10526   69.157659  6300      1
10527  199.475985  5800      0
10528   50.665847  5800      1
10529   78.973925  5800      0

[10530 rows x 3 columns]


In [36]:
final_df.to_csv('modelData_1.csv', index=False)